<div style='float: right'><img src='pic/hitori.png'/></div>

## <div id='hitori' />ひとりにしてくれ

In [1]:
#!pip3 install pulp ortoolpy
import numpy as np
from itertools import chain, product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars, unionfind
data = np.array([list(s) for s in """\
18626753
31118222
83247651
37583314
54467182
71432535
22834475
22314465""".split()]).astype(int)
nn = len(data)

### 問題
* 盤面に並んでいる数字のうちいくつかを黒くぬり、タテでもヨコでも同じ列に同じ数字が複数個入らないようにします
* 黒マスをタテヨコに連続したり、黒マスで盤面を分断してはいけません

### 変数
* v：0:number, 1:black (1)

### 制約
* 黒以外の数字は縦横に重複しないこと (2)
* 黒は連続しないこと (3)
* 黒で分断しないこと (4)

In [2]:
m = LpProblem()
v = np.array(addbinvars(nn, nn)) # 0:number, 1:black (1)
for i, j in product(range(nn), range(nn)):
    for x in [v[i,:][data[i,:] == j+1], v[:,i][data[:,i] == j+1]]:
        m += lpSum(x) >= len(x)-1 # (2)
for e in chain((v[1:,:] + v[:-1,:]).flat, (v[:,1:] + v[:,:-1]).flat):
    m += e <= 1 # (3)
while True:
    %time m.solve()
    r = np.vectorize(value)(v)
    if unionfind.isconnected(1-r):
        break
    m += lpSum(v[r==1]) <= r.sum() - 1 # (4)
data[r==1] = 0
data.tolist()

CPU times: user 8.02 ms, sys: 5.04 ms, total: 13.1 ms
Wall time: 32.8 ms
CPU times: user 11.2 ms, sys: 5.7 ms, total: 16.9 ms
Wall time: 44.7 ms
CPU times: user 7.95 ms, sys: 4.69 ms, total: 12.6 ms
Wall time: 36.4 ms
CPU times: user 8.53 ms, sys: 5.27 ms, total: 13.8 ms
Wall time: 30.7 ms
CPU times: user 9 ms, sys: 5.33 ms, total: 14.3 ms
Wall time: 30.5 ms
CPU times: user 10.3 ms, sys: 4.57 ms, total: 14.8 ms
Wall time: 30 ms
CPU times: user 14.4 ms, sys: 6.7 ms, total: 21.1 ms
Wall time: 45.9 ms
CPU times: user 13.7 ms, sys: 5.66 ms, total: 19.3 ms
Wall time: 40 ms
CPU times: user 8.57 ms, sys: 5.31 ms, total: 13.9 ms
Wall time: 31.5 ms


[[1, 8, 0, 2, 6, 7, 5, 3],
 [3, 0, 1, 0, 8, 0, 2, 0],
 [8, 3, 2, 4, 7, 6, 0, 1],
 [0, 7, 5, 8, 3, 0, 1, 4],
 [5, 4, 0, 6, 0, 1, 8, 2],
 [7, 1, 4, 0, 2, 5, 3, 0],
 [2, 0, 8, 3, 4, 0, 7, 5],
 [0, 2, 3, 1, 0, 4, 6, 0]]